### Using ExplainReduce on the Auto MPG dataset
In this notebook we will demonstrate how to use ExplainReduce on the Auto MPG dataset. We use Slisemap as the underlying local model generation method.

In [1]:
import sys

import numpy as np
import pandas as pd

from pathlib import Path
from urllib.request import urlretrieve

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

sys.path.insert(0, "..")

from slisemap import Slisemap

#### Model caching
To reduce execution times, we provide dumps of pretrained Slisemap models. If you want to train them yourself instead please set USE_CACHE=False.

In [2]:
USE_CACHE = False
SM_CACHE_PATH = Path("cache") / "01_regression_example_autompg.sm"

if USE_CACHE:
    for path in [SM_CACHE_PATH]:
        path.parent.mkdir(exist_ok=True, parents=True)
        if not path.exists():
            urlretrieve(
                f"https://raw.githubusercontent.com/edahelsinki/slisemap/data/examples/cache/{path.name}",
                path,
            )

#### Data
The Auto MPG dataset is a multivariate real-valued dataset with eight attributes describing fuel consumption -related properties of 398 distinct automobiles. We use mpg (miles per gallon) as the target variable and we have additionally removed 6 data items that had missing values. The data consists of 3 discrete and 5 continuous attributes (one of which is mpg) and it is available through UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/auto+mpg

In [3]:
"""
Load the AutoMPG dataset (download it if necessary).
Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml].
 Irvine, CA: University of California, School of Information and Computer Science.
Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning.
 In Proceedings on the Tenth International Conference of Machine Learning, 236-243,
 University of Massachusetts, Amherst. Morgan Kaufmann.
"""
path = Path("data") / "auto-mpg.data"
path.parent.mkdir(exist_ok=True, parents=True)
if not path.exists():
    urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data",
        path,
    )
auto_mpg = pd.read_csv(
    path,
    names=[
        "mpg", "cylinders", "displacement", "horsepower", "weight",
        "acceleration", "year", "origin", "carname",
    ],
    delim_whitespace=True,
    na_values=["?"],
)

C:\Users\Localadmin_mudonggu\AppData\Local\Temp\ipykernel_13772\3030633475.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  auto_mpg = pd.read_csv(


In [4]:
X0 = auto_mpg[[
    "cylinders", "displacement", "horsepower", "weight", "acceleration", "year", "origin",
]]
y0 = auto_mpg["mpg"]

# Split and one-hot encode the origin into USA vs Europe vs Japan
X0 = np.concatenate(
    (X0.values[:, :-1].astype(float), np.eye(3)[X0["origin"].values.astype(int) - 1]), axis=1,
)
y0 = y0.values

# X0 contains the covariates, y0 is the target variable and names are column names.
mask = ~np.isnan(X0[:, 2])
X0 = X0[mask]
y0 = y0[mask]

names = list(auto_mpg.columns[1:-2]) + ["origin USA", "origin Europe", "origin Japan"]

In [5]:
pd.DataFrame(np.concatenate([y0.reshape((-1,1)), X0], axis=1), columns=["mpg"]+names)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin USA,origin Europe,origin Japan
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,0.0,0.0
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,0.0,0.0
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,0.0,0.0
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,0.0,0.0
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
387,27.0,4.0,140.0,86.0,2790.0,15.6,82.0,1.0,0.0,0.0
388,44.0,4.0,97.0,52.0,2130.0,24.6,82.0,0.0,1.0,0.0
389,32.0,4.0,135.0,84.0,2295.0,11.6,82.0,1.0,0.0,0.0
390,28.0,4.0,120.0,79.0,2625.0,18.6,82.0,1.0,0.0,0.0


In [6]:
# X and y are normalised by `sklearn.preprocessing.StandardScaler`.
scale_x = StandardScaler()
scale_y = StandardScaler()
X = np.concatenate([scale_x.fit_transform(X0[:, :-3]), X0[:, -3:]], axis=1)
y = scale_y.fit_transform(y0[:, None])
# We also remove ten random datapoints from the data for testing later.
X, X_test, y, y_test = train_test_split(X, y, test_size=7, random_state=42)

#### Black box model
Instead of simply running Slisemap on the data itself, we showcase how it can be utilised to provide explanations on a black box model. Typically a black box model is produced via some machine learning algorithm and the inner workings of the resulting model are too complicated for a human to understand. Here we train and use a SVM from sklearn.svm.

In [7]:
svr = SVR().fit(X, y.ravel())
y2 = svr.predict(X)

#### Slisemap
Use a Slisemap explainer to generate local models for the black-box model.

In [8]:
import torch
import explainreduce.localmodels as lm 

# Use a slisemap explainer
sm_explainer = lm.SLISEMAPExplainer(
    X=torch.tensor(X, dtype=torch.float32),
    y=torch.tensor(y2, dtype=torch.float32),
    lasso=0.01,
)
sm_explainer.fit()

c:\Users\Localadmin_mudonggu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here for each item in the training set (totally 385 items), we have a corresponding linear local model.

In [9]:
sm_explainer.vector_representation, sm_explainer.vector_representation.shape

(tensor([[-1.2509e-05,  7.0450e-06, -2.5707e-01,  ...,  5.4793e-06,
           4.7097e-02, -7.6715e-06],
         [-1.7261e-01, -1.1321e-01,  1.5296e-06,  ...,  1.3028e-05,
           1.0497e-05, -7.4485e-06],
         [ 3.9120e-06, -1.0666e-05, -2.5718e-01,  ...,  1.1157e-05,
           4.6902e-02, -2.4783e-06],
         ...,
         [-1.7823e-01, -2.1140e-05, -2.5871e-05,  ..., -1.5368e-05,
           9.4526e-07, -4.1667e-05],
         [-2.3847e-05, -1.3585e-05, -2.5701e-01,  ...,  8.0615e-07,
           4.7468e-02, -4.2799e-06],
         [-1.7791e-01, -1.6620e-04, -1.1761e-05,  ..., -1.4434e-05,
           2.0298e-05, -2.9571e-05]]),
 torch.Size([385, 10]))

The loss matrix aggregates the loss of applying each local model on every item in the training set. For example, the first row is the loss of using the local model of the first item to predict the whole training set. The average loss is 0.1743.

In [10]:
sm_explainer.get_L(), sm_explainer.get_L().shape, sm_explainer.get_L().mean()

(tensor([[4.2254e-06, 7.5176e-02, 1.1192e-04,  ..., 8.8568e-02, 6.6603e-07,
          4.6529e-01],
         [1.3241e-01, 7.5693e-04, 1.9821e-01,  ..., 3.9171e-03, 6.8754e-01,
          3.0139e-02],
         [3.6355e-06, 7.5328e-02, 1.1311e-04,  ..., 8.8746e-02, 2.0069e-07,
          4.6494e-01],
         ...,
         [1.1518e-01, 2.9021e-02, 1.2507e-01,  ..., 3.9614e-03, 4.4132e-01,
          3.0452e-03],
         [6.5626e-06, 7.5091e-02, 1.0721e-04,  ..., 8.8661e-02, 1.4181e-07,
          4.6589e-01],
         [1.1518e-01, 2.8824e-02, 1.2485e-01,  ..., 3.8463e-03, 4.4236e-01,
          3.0747e-03]]),
 torch.Size([385, 385]),
 tensor(0.1743))

#### Apply reduction to get the proxies
Here we use the greedy loss reduction method, with expected proxies number set to 5 and coverage expectation to 80%.

In [11]:
import explainreduce.proxies as px
reduced_sm_explainer = px.find_proxies_greedy_k_min_loss(
    explainer=sm_explainer,
    k=5,
)

Now we have a reduced explanation set with 5 proxies, the average loss is 0.1795, very close to the original explanation set.

In [12]:
reduced_sm_explainer.vector_representation, reduced_sm_explainer.vector_representation.shape

(tensor([[-8.8547e-06, -4.9765e-02, -1.8553e-02, -6.3732e-01,  1.9076e-06,
           3.2872e-01, -1.5689e-01, -8.6860e-05,  1.6822e-02, -2.5269e-04],
         [ 5.6257e-02,  9.3633e-02, -5.6361e-01, -6.3810e-01,  8.3662e-02,
           3.6969e-01,  6.6815e-06, -4.5865e-03,  7.7091e-06, -9.9494e-02],
         [-1.7224e-01, -1.1353e-01, -1.1800e-05, -2.6111e-01, -7.0723e-02,
           1.9629e-01, -3.3596e-01,  1.4391e-05,  1.4776e-06, -1.3858e-04],
         [-1.7675e-01,  5.7212e-06, -8.5173e-06, -3.5556e-01, -1.4104e-02,
           1.5552e-01, -1.8900e-01, -4.9126e-06,  1.7652e-05, -4.5481e-03],
         [ 1.0184e-05, -1.0307e-06, -4.0093e-01, -5.2470e-01,  8.5111e-06,
           3.9684e-01, -6.0900e-02, -1.6036e-05,  2.3373e-02, -4.2997e-07]]),
 torch.Size([5, 10]))

In [13]:
reduced_sm_explainer.get_L(), reduced_sm_explainer.get_L().shape, reduced_sm_explainer.get_L().mean()

(tensor([[1.4479e-02, 1.7692e-02, 2.4911e-02,  ..., 1.7085e-02, 6.7794e-02,
          4.6061e-01],
         [8.4502e-03, 1.4985e-01, 1.5279e-01,  ..., 1.9480e-01, 1.5140e-01,
          9.6599e-01],
         [1.3248e-01, 7.8132e-04, 1.9829e-01,  ..., 3.8762e-03, 6.8701e-01,
          3.0390e-02],
         [1.2086e-01, 3.6374e-02, 1.1913e-01,  ..., 4.9810e-03, 4.4925e-01,
          1.4908e-03],
         [3.3074e-03, 1.2810e-01, 1.7278e-02,  ..., 1.7730e-01, 3.8489e-02,
          5.2490e-01]]),
 torch.Size([5, 385]),
 tensor(0.1795))

#### Use the proxies to make predictions
We have 7 test items, below we show the predictions on those items given by the black-box SVR, the full set of local explanations, and the proxies respectively. 

In [29]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
print(f"Predictions from the black box: {svr.predict(X_test_tensor)}")
print(f"Predictions from the full set: {sm_explainer.predict(X_test_tensor).ravel().detach().numpy()}")
print(f"Predictions from the proxies: {reduced_sm_explainer.predict(X_test_tensor).ravel().detach().numpy()}")

Predictions from the black box: [ 0.38712107 -0.12080159  1.43031837  0.03801568  0.64973534  0.75055089
 -1.35231908]
Predictions from the full set: [ 0.3100698  -0.14047301  1.4497539  -0.15255666  0.53849864  0.6774812
 -1.4204621 ]
Predictions from the proxies: [ 0.30051297 -0.02767373  1.4334548  -0.15312657  0.55653054  0.6969903
 -1.3894179 ]
